In [78]:
import yfinance as yf
from scipy.stats import norm, zscore, halfnorm
import numpy as np
import pandas as pd
import math
import scipy as sq
import yahoo_fin.stock_info as si
from datetime import datetime, timedelta, date
import os
pd.options.display.float_format = '{:.4f}'.format


def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [61]:
# ticker = yf.Ticker("ASO")
# todays_data = ticker.history(period='1y')
# todays_data

In [62]:
def get_current_stock_price(symbol):
    tk = yf.Ticker(symbol)
    return tk.history(period='1d')['Close'][0]

def get_days_to_expiration(symbol):
    tk = yf.Ticker(symbol)
    exps = tk.options
    e = exps[0]
    dt_obj = datetime.strptime(e, '%Y-%m-%d')
    days_left = abs((datetime.now() - dt_obj).days)
    return days_left

def get_earliest_deadline_options_chain(symbol):
    """ Get call and put options for earliest deadline
    """
    tk = yf.Ticker(symbol)
    exps = tk.options
    options = pd.DataFrame()
    e = exps[0] 

    opt = tk.option_chain(e)
    calls = pd.DataFrame(opt.calls)
    puts = pd.DataFrame(opt.puts)
    calls['expirationDate'] = e
    puts['expirationDate'] = e
    
    options = pd.concat([options, calls])
    options = pd.concat([options, puts])
    
    options['isCall'] = options['contractSymbol'].str[4:].apply(lambda x: "C" in x)
    options[['bid', 'ask', 'strike']] = options[['bid', 'ask', 'strike']].apply(pd.to_numeric)
    options['mark'] = (options['bid'] + options['ask']) / 2
    options = options.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate'])
    
#     vol = []
#     for row in options.iterrows():
#         S = get_current_price(symbol)
#         X = row[1].strike
#         r = 0.03
#         T = get_days_to_expiration(symbol)
#         option = row[1].mark
#         p_c = 'C'
#         if row[1].isCall == False :
#             p_c = 'P'
            
#         IV = Calculate_IV_call_Put(S, X, r, T, option, p_c)
#         vol.append(IV)
    
#     options["Black Scholes IV"] = vol
    return options

# get_earliest_deadline_options_chain("FB")

In [84]:
def get_expected_move(symbol):
    """Get Expected Move for a stock for the earliest option deadline. Returns tuple
    [0] - IV Expected Move
    [1] - ATM Straddle Expected Move
    [2] - ATM Straddle + 1st Strangle Expected Move
    [3] - Aggregate Estimate of Stock Expected Move (Used in calculations)
    """
    stock = yf.Ticker(symbol)
    curr = stock.history(period='1d')['Close'][0]
    options = get_earliest_deadline_options_chain(symbol)
        
    test_list = list(set(options.strike))
    temp = sorted(test_list, key=lambda x:abs(x-curr))
    if len(temp) < 3:
        return "Error"
    
    minVal = temp[0]
    higherVal = temp[1] if temp[1] > temp[2] else temp[2]
    lowerVal = temp[2] if temp[1] > temp[2] else temp[1]
    
    strad = options.loc[options.strike == minVal]
    low_stran = options.loc[options.strike == lowerVal]
    high_stran = options.loc[options.strike == higherVal]
    
    # IV Expected Move = Stock Price x (Implied Volatility / 100) x square root of (Days to Expiration / 365)
    expected_move = curr * strad.impliedVolatility * math.sqrt(get_days_to_expiration(symbol)/365)
    per = expected_move / curr
    
    straddle_sum = 0
    if len(strad.mark) == 2:
        straddle_sum = strad.mark.sum()
    elif len(strad.mark) == 1:
        straddle_sum = strad.mark.sum() * 2

    strangle_sum = 0
    high_mark = high_stran.loc[high_stran.isCall == True].mark
    low_mark = low_stran.loc[low_stran.isCall == False].mark
    if not high_mark.empty and not low_mark.empty:
        strangle_sum = high_mark.iloc[0] + low_mark.iloc[0]
    
    # If mark price or bid/ask price is 0, use last price to calculuate straddle and strangle Move
    if straddle_sum == 0:
        if len(strad.lastPrice) == 2:
            straddle_sum = strad.lastPrice.sum()
        elif len(strad.lastPrice) == 1:
            straddle_sum = strad.lastPrice.sum() * 2
    if strangle_sum == 0:
        high_last = high_stran.loc[high_stran.isCall == True].lastPrice
        low_last = low_stran.loc[low_stran.isCall == False].lastPrice
        if not high_last.empty and not low_last.empty:
            strangle_sum = high_last.iloc[0] + low_last.iloc[0]

    # Straddle Expected Move = 70% of ATM straddle + 30% of 1st Strangle
    total = 0.7 * (straddle_sum) + 0.3 * (strangle_sum)
    
    
    # If IV Move is too low, only straddle move
    avg = (per.mean() + (straddle_sum / curr)) / 2
    if(per.mean() <= 0.01):
        avg = straddle_sum / curr
    
    # IV Expected, ATM Straddle, ATM Straddle + 1st Strangle, Avg of IV + ATM
    return (per.mean(), straddle_sum / curr, total / curr, avg)

# get_expected_move("ORCL")

In [83]:
def get_put_option_chain(symbol, price_ceiling = -1):
    """ Get only the put option chain. Defaults to finding only ITM puts. Set price_ceiling to set cutoff 
    """
    tk = yf.Ticker(symbol)
    exps = tk.options
    e = exps[0] 

    opt = tk.option_chain(e)
    puts = pd.DataFrame(opt.puts)
    
    curr_price = tk.history(period='1d')['Close'][0]

    puts[['bid', 'ask', 'strike']] = puts[['bid', 'ask', 'strike']].apply(pd.to_numeric)
    puts.insert(6, 'mark', (puts['bid'] + puts['ask']) / 2)
    puts.insert(12, 'dist_to_stock%', abs(((curr_price - puts['strike'] ) / curr_price) * 100))
    puts = puts.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate', 'inTheMoney'])
    
    moves = get_expected_move(symbol)
    puts['aggregate_move%'] = moves[3] * 100
    puts['IV_move%'] = moves[0] * 100
    puts['straddle_move%'] = moves[1] * 100
    puts['strad_strang_move%'] = moves[2] * 100
    puts['expirationDate'] = e
    
    if price_ceiling == -1:
        price_ceiling = curr_price
    
    index_names = puts[(puts['strike'] > price_ceiling)].index
    puts = puts.drop(index_names)    
    
    zscore = (puts['dist_to_stock%'] / puts['aggregate_move%']) 
    puts.insert(6, '%probability_not_called', (halfnorm.cdf(zscore) * 100).round(4))
    collateral = puts['mark'] * 100
    if(collateral.all() == 0):
        collateral = puts['lastPrice'] * 100
    puts.insert(7, '%gain_of_collateral', (collateral * 100 / puts['strike']))    
    
    puts.sort_values(by=['strike'], inplace=True, ascending=False)
    puts.reset_index(drop=True, inplace=True)
    return puts

# get_put_option_chain("DOCU")

In [65]:
def get_all_put_options(symbol):
    tk = yf.Ticker(symbol)
    exps = tk.options
    e = exps[0] 

    opt = tk.option_chain(e)
    puts = pd.DataFrame(opt.puts)
    
    curr_price = tk.history(period='1d')['Close'][0]

    puts[['bid', 'ask', 'strike']] = puts[['bid', 'ask', 'strike']].apply(pd.to_numeric)
    puts.insert(6, 'mark', (puts['bid'] + puts['ask']) / 2)
    puts.insert(12, 'dist_to_stock%', abs(((curr_price - puts['strike'] ) / curr_price) * 100))
    
    puts = puts.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate'])

    
    moves = get_expected_move(symbol)
    puts['aggregate_move%'] = moves[3] * 100
    puts['IV_move%'] = moves[0] * 100
    puts['straddle_move%'] = moves[1] * 100
    puts['strad_strang_move%'] = moves[2] * 100
    
    puts['expirationDate'] = e
     
    puts.sort_values(by=['strike'], inplace=True, ascending=False)
    puts.reset_index(drop=True, inplace=True)
    
    zscore = (puts['dist_to_stock%'] / puts['aggregate_move%']) 
    puts.insert(6, '%probability_not_called', (halfnorm.cdf(zscore) * 100).round(4))
    collateral = puts['mark'] * 100
    if(collateral.all() == 0):
        collateral = puts['lastPrice'] * 100
        
    puts.insert(7, '%gain_of_collateral', (collateral * 100 / puts['strike']))   
    return puts

#get_all_put_options("AAPL")

In [82]:
def get_stock_earnings_price_effect(symbol, period = "5y"):
    """ Get Earnings Price Effect. Returns Pandas DataFrame with Data of Market Close to Next Day Open and Close
    Set period to set cutoff
    """
    stock = yf.Ticker(symbol)
    hist = stock.history(period)
    earn_hist = si.get_earnings_history(symbol)
    data = {}
    is_AMC = earn_hist[0]['startdatetimetype'] == "AMC" #Default AMC
    for earning in earn_hist: 
        earn_date = earning['startdatetime']
        dt_obj = datetime.strptime(earn_date, '%Y-%m-%dT%H:%M:%S.000Z')
        date = dt_obj.date()
        
        # If hour is < 10, count as BMO, else count as AMC
        if(dt_obj.hour < 10): 
            is_AMC = False
        else:
            is_AMC = True

        if is_AMC:
            earning_day = hist.loc[(hist.index == str(date))]
            earning_next_day = hist.loc[hist.index == str(date + timedelta(days = 1))]
            if not earning_day.empty and not earning_next_day.empty:
                earning_effect_close_to_next_open = earning_next_day["Open"].iloc[0] - earning_day["Close"].iloc[0]
                earning_effect_close_to_next_close = earning_next_day["Close"].iloc[0] -  earning_day["Close"].iloc[0]

                data[str(date)] = [earning_effect_close_to_next_open, 
                                  earning_effect_close_to_next_open * 100 / earning_day["Close"].iloc[0] ,
                                   earning_effect_close_to_next_close ,
                                  earning_effect_close_to_next_close * 100 / earning_day["Close"].iloc[0]]

        else:
            earning_day = hist.loc[(hist.index == str(date))]
            earning_prev_day = hist.loc[hist.index == str(date - timedelta(days = 1))]
            if not earning_day.empty and not earning_prev_day.empty:
                earning_effect_prev_close_to_open = earning_day["Open"].iloc[0] -  earning_prev_day["Close"].iloc[0]
                earning_effect_prev_close_to_close = earning_day["Close"].iloc[0] -  earning_prev_day["Close"].iloc[0]

                data[str(date)] = [earning_effect_prev_close_to_open,
                                  earning_effect_prev_close_to_open * 100/ earning_prev_day["Close"].iloc[0] ,
                                   earning_effect_prev_close_to_close,
                                  earning_effect_prev_close_to_close * 100/ earning_prev_day["Close"].iloc[0]]

    earn_data = pd.DataFrame.from_dict(data, orient='index',
                columns=['day_close_to_next_day_open', '%change_close_to_next_open',
                        'day_close_to_next_day_close', '%change_close_to_next_close'])
    
    new_row = pd.DataFrame({'day_close_to_next_day_open':earn_data['day_close_to_next_day_open'].abs().mean(), '%change_close_to_next_open':earn_data['%change_close_to_next_open'].abs().mean(), 
                            'day_close_to_next_day_close':earn_data['day_close_to_next_day_close'].abs().mean(), '%change_close_to_next_close':earn_data['%change_close_to_next_close'].abs().mean()}, 
                           index=["Absolute Average"])

    earn_data = pd.concat([earn_data, new_row])
    earn_data.insert(0, 'Symbol', symbol)

    return earn_data

# get_stock_earnings_price_effect("ORCL")

In [102]:
def get_stocks_with_weekly_options():
    """ Returns set of all stocks with weekly options"""
    weeklyOptions = pd.read_csv(os.path.join('input_data','cboesymboldirweeklys.csv'))
    weeklyOptions.columns = ['name', 'symbol']
    weeklySet = set(weeklyOptions['symbol'])
    return weeklySet
    
def get_stocks_with_options():
    """ Returns set of all stocks with options"""
    indexOptions = pd.read_csv(os.path.join('input_data','cboesymboldirequityindex.csv'))
    indexOptions.drop(columns = [' DPM Name', ' Post/Station'], inplace=True)
    indexOptions.columns = ['name', 'symbol']
    optionSet = set(indexOptions['symbol'])
    return optionSet

In [158]:
def get_companies_with_earnings_today_AMC_or_tomm_BMO(add_days = 0, min_vol_sum = 3000, min_stock_price = 10):
    curr_date = datetime.now().date() +  timedelta(add_days)
        
    earn_today = si.get_earnings_for_date(str(curr_date))
    earn_tomm = si.get_earnings_for_date(str(curr_date + timedelta(1)))
    
    comp_today = pd.DataFrame.from_dict(earn_today)
    comp_tomm = pd.DataFrame.from_dict(earn_tomm)
        
    if comp_today.empty and comp_tomm.empty:
        return pd.DataFrame()
    
    if not comp_today.empty:
        comp_today['startdatetime'] = pd.to_datetime(comp_today['startdatetime'])
        index_names = comp_today[(comp_today['startdatetime'].dt.hour < 10)].index
        comp_today = comp_today.drop(index_names)
        comp_today['startdatetimetype'] = 'AMC'

    if not comp_tomm.empty:
        comp_tomm['startdatetime'] = pd.to_datetime(comp_tomm['startdatetime'])
        index_names = comp_tomm[(comp_tomm['startdatetime'].dt.hour >= 10)].index
        comp_tomm = comp_tomm.drop(index_names)
        comp_tomm['startdatetimetype'] = 'BMO'
        
    comp = pd.DataFrame()
    comp = pd.concat([comp_today, comp_tomm])
    comp = comp.drop(columns = ['gmtOffsetMilliSeconds', 'quoteType', 'epsactual', 'epssurprisepct'])
    comp = comp.drop_duplicates()
    comp.reset_index(drop=True, inplace=True)

    comp.insert(2, "stock_price", 0)
    comp.insert(3, "is_weekly", False)
    comp['volume'] = 0
    
    optionSet = get_stocks_with_options()
    weeklySet = get_stocks_with_weekly_options()
    for row in comp.iterrows():
        ticker = row[1].ticker
        if ticker not in optionSet:
            continue

        tk = yf.Ticker(ticker)
        exps = tk.options        
        if not exps: 
            continue

        e = exps[0]
        opt = tk.option_chain(e)
        volume_sum = opt.calls.volume.sum() + opt.puts.volume.sum()
        
        if volume_sum < min_vol_sum:
            continue
            
        openInterestSum = opt.calls.openInterest.sum() + opt.puts.openInterest.sum()

        rowVal = comp.index[comp['ticker'] == ticker]
        comp.loc[rowVal, 'stock_price'] = round(tk.history(period='1d')['Close'][0] , 2)
        comp.loc[rowVal, 'volume'] = volume_sum
        comp.loc[rowVal, 'openInterest'] = openInterestSum
        comp.loc[rowVal, 'is_weekly'] = (ticker in weeklySet)    
        
        move = get_expected_move(ticker)
        comp.loc[rowVal, 'aggregate_move%'] = move[3] * 100

    index_names = comp[(comp['volume'] == 0) | (comp['stock_price'] <= min_stock_price)].index
    comp = comp.drop(index_names)

    comp.sort_values(by=['volume'], inplace=True, ascending=False)
    comp.reset_index(drop=True, inplace=True)

    return comp

# today_comp = get_companies_with_earnings_today_AMC_or_tomm_BMO(-1)
# print(today_comp)
# today_comp.to_html("today_companies.html")

  ticker    companyshortname  stock_price  is_weekly  \
0   ORCL  Oracle Corporation      64.0500       True   

              startdatetime startdatetimetype  epsestimate timeZoneShortName  \
0 2022-06-13 16:05:00+00:00               AMC       1.3700               EDT   

   volume  openInterest  aggregate_move%  
0  131034      588.0000           9.5863  


In [159]:
def get_IV_crush_for_puts():
    """ Calculuate Previous day's IV crush"""
    
    dir = os.path.join(str(date.today().year), str(date.today() + timedelta(-1)))

    if not os.path.exists(dir):
        print("Directory does not exist for IV Crush")
        return
    
    num_files = len([name for name in os.listdir(dir) if os.path.isfile(os.path.join(dir, name))])

    for x in range(num_files - 1):
        list_companies = pd.read_csv(os.path.join(dir, "companies.csv"))
        tickers = list_companies[['ticker', 'stock_price']]

        company = pd.read_csv(os.path.join(dir, tickers['ticker'][x] + '.csv'))
        df = get_put_option_chain(tickers['ticker'][x], tickers['stock_price'][x])[['strike','impliedVolatility']]
        # Calculuate IV for ATM and feasible OTM options
        ls = company[company['%probability_not_called'] != 100]['impliedVolatility']
        company['IV_crush'] = (df['impliedVolatility'] - ls)/ ls
        company['nextday_impliedVolatility'] = df['impliedVolatility']
        avg = company['IV_crush'].mean()
        company.loc['Average'] = {'IV_crush': avg}
        
        company.to_csv(os.path.join(dir, tickers['ticker'][x] + '.csv'), index=False)
        
# get_IV_crush_for_puts()

In [163]:
def check_today(duration = "10y", days_added = 0):
    today_comp = get_companies_with_earnings_today_AMC_or_tomm_BMO(days_added)
    total = pd.DataFrame()
    price = pd.DataFrame()
    outdir = os.path.join(str(date.today().year), str(date.today()))

    for row in today_comp.iterrows():
        ticker = row[1].ticker
        temp = get_put_option_chain(ticker)

        if not os.path.exists(outdir):
            os.mkdir(outdir)

        fullname = os.path.join(outdir, ticker + ".csv")    
        temp.to_csv(fullname)
        
        today_path = os.path.join(outdir, "companies.csv")    
        today_comp.to_csv(today_path)

        total = pd.concat([total, temp], ignore_index=True)
        total.loc["Empty"] = '' 
        total.loc["Space"] = ''        

        price_effect = get_stock_earnings_price_effect(ticker, duration)
        price_effect.loc["Empty"] = '' 
        price_effect.loc["Space"] = '' 
        price = pd.concat([price, price_effect])   
    
    today_comp.to_html("today_companies.html")
    total.to_html('today_options.html')
    price.to_html('today_price_effect.html')
    get_IV_crush_for_puts()
    print("Done")
    
    return (total, price)

In [164]:
check_today("4y", -1)

Done


(            contractSymbol  strike lastPrice    bid    ask   mark  \
 0      ORCL220617P00064000 64.0000    3.1000 0.0000 0.0000 0.0000   
 1      ORCL220617P00063000 63.0000    2.7000 0.0000 0.0000 0.0000   
 2      ORCL220617P00062500 62.5000    2.5400 0.0000 0.0000 0.0000   
 3      ORCL220617P00062000 62.0000    2.2500 0.0000 0.0000 0.0000   
 4      ORCL220617P00061000 61.0000    1.9900 0.0000 0.0000 0.0000   
 5      ORCL220617P00060000 60.0000    1.6800 0.0000 0.0000 0.0000   
 6      ORCL220617P00059000 59.0000    1.4300 0.0000 0.0000 0.0000   
 7      ORCL220617P00058000 58.0000    1.2000 0.0000 0.0000 0.0000   
 8      ORCL220617P00057000 57.0000    1.0300 0.0000 0.0000 0.0000   
 9      ORCL220617P00056000 56.0000    0.8700 0.0000 0.0000 0.0000   
 10     ORCL220617P00055000 55.0000    0.7200 0.0000 0.0000 0.0000   
 11     ORCL220617P00050000 50.0000    0.3000 0.0000 0.0000 0.0000   
 12     ORCL220617P00047500 47.5000    0.1800 0.0000 0.0000 0.0000   
 13     ORCL220617P0